# Git Going
## An intermediate tutorial on git

### What this tutorial is
- An introduction to the git data model.
- An exposition of the core DAG operations.
- An introduction to collaboration with git remotes.
- A recommended workflow and other tips.

### What this tutorial isn't
- A tutorial on the basics. You should be familiar with:
    - `git init`
    - `git add`
    - `git commit`
    - `git clone`
    - `git checkout`

## What is git?
- A distributed version control system\* (but you knew that).
- A **directed acyclic graph** of commits and references.
- A **commit** is a diff between its own state and the state of its parent(s).
- A **reference** is a pointer to a commit.
- Two main types of references: **branches** and **tags**.
- References are mutable; commits are not.

<small>
\*At its lowest level, git implements a *content addressable file system* capable of supporting arbitrary data structures and applications. Version control is the primary use case, but not the only one.
</small>

## In the beginning

When a git repository is created:
- there are no commits
- there is a single reference, the master branch, which points to nothing.

In [1]:
git init

Initialized empty Git repository in /Users/csb/Desktop/gitgoing/.git/


In [2]:
git log --oneline; true  # use `true` to swollow the non-zero exit

fatal: your current branch 'master' does not have any commits yet


![only master](images/00.0-master.svg)

Let's create an initial commit:

In [3]:
echo '0. Initial commit' >> master.txt
git add master.txt
git commit -m 'Initial commit'

[master (root-commit) 553dbb7] Initial commit
 1 file changed, 1 insertion(+)
 create mode 100644 master.txt


In [4]:
git log --oneline

553dbb7 (HEAD -> master) Initial commit


![initial commit](images/00.1-init.svg)

## Creating references with `git branch`

The `branch` command creates a new branch pointing to the current commit.

The `checkout -b` command creates a new branch and immediately checks it out.

In [5]:
git branch foo

In [6]:
git branch -v

  foo    553dbb7 Initial commit
* master 553dbb7 Initial commit


![branch foo](images/01-foo.svg)

In [7]:
git checkout -b bar

Switched to a new branch 'bar'


In [8]:
git branch -v

* bar    553dbb7 Initial commit
  foo    553dbb7 Initial commit
  master 553dbb7 Initial commit


![git checkout -b bar](images/02-bar.svg)

## Moving branches with `git reset`

The `reset` command moves the current branch to a different commit.

In [9]:
echo '1. commit on branch bar' >> bar.txt
git add bar.txt
git commit -am 'Make commit on branch bar'

[bar ac05751] Make commit on branch bar
 1 file changed, 1 insertion(+)
 create mode 100644 bar.txt


In [10]:
git log --oneline

ac05751 (HEAD -> bar) Make commit on branch bar
553dbb7 (master, foo) Initial commit


![git commit on bar](images/04-commit-on-bar.svg)

In [11]:
echo '2. commit on branch bar, again' >> bar.txt
git commit -am 'Make commit on branch bar, again'

[bar e5ceb5e] Make commit on branch bar, again
 1 file changed, 1 insertion(+)


In [12]:
git log --oneline

e5ceb5e (HEAD -> bar) Make commit on branch bar, again
ac05751 Make commit on branch bar
553dbb7 (master, foo) Initial commit


![git commit on bar again](images/05-commit-on-bar-again.svg)

In [13]:
git reset --hard HEAD~1

HEAD is now at ac05751 Make commit on branch bar


In [14]:
git log --oneline

ac05751 (HEAD -> bar) Make commit on branch bar
553dbb7 (master, foo) Initial commit


`HEAD` is a special reference that always points to the current commit. The syntax `REF~1` means "the parent of `REF`.

![git reset](images/06-reset.svg)

You may have noticed that `cff7ce4` is unreachable from any branch. It's _garbage_. In fact, git has a garbage collector! However, the garbage collector is not automatic, since you can reach any commit directly via its hash. It's generally not useful to delete commits, but if you're low on storage, checkout the `git gc` command.

## Commit with multiple parents using `git merge`

The `merge` command creates commits with two or more parents. merge commits contain a diff for all parents to the new combined state.

In [15]:
git checkout foo
echo '3. commit on branch foo' >> foo.txt
git add foo.txt
git commit -am 'Make commit on branch foo'

Switched to branch 'foo'
[foo b43daa9] Make commit on branch foo
 1 file changed, 1 insertion(+)
 create mode 100644 foo.txt


In [16]:
git log --oneline

b43daa9 (HEAD -> foo) Make commit on branch foo
553dbb7 (master) Initial commit


![commit on foo](images/07-commit-foo.svg)

In [17]:
git merge bar -m "Merge branch 'bar' into 'foo'"

Merge made by the 'recursive' strategy.
 bar.txt | 1 +
 1 file changed, 1 insertion(+)
 create mode 100644 bar.txt


In [18]:
git log --oneline

93ba65d (HEAD -> foo) Merge branch 'bar' into 'foo'
b43daa9 Make commit on branch foo
ac05751 (bar) Make commit on branch bar
553dbb7 (master) Initial commit


![merge bar into foo](images/08-merge-bar-into-foo.svg)

## Moving commits with `git rebase`

The `rebase` command simulates the moving of commits.

To rebase `foo` onto `master` means to take all commits between `foo` and the neares common ancestor with `master` and move them on top of master.

Commits are immutable, so we can't move them per se. Instead, we make copies. And while we're at it, we linearize the history.

In [19]:
git checkout master
echo '4. commit on branch master' >> master.txt
git commit -am 'Make commit on branch master'

Switched to branch 'master'
[master 12844aa] Make commit on branch master
 1 file changed, 1 insertion(+)


In [20]:
git log --oneline

12844aa (HEAD -> master) Make commit on branch master
553dbb7 Initial commit


![commit on master](images/09-commit-on-master.svg)

In [21]:
git checkout foo
git rebase master

Switched to branch 'foo'
First, rewinding head to replay your work on top of it...
Applying: Make commit on branch bar
Applying: Make commit on branch foo


In [22]:
git log --oneline

0e9470e (HEAD -> foo) Make commit on branch foo
e33d14f Make commit on branch bar
12844aa (master) Make commit on branch master
553dbb7 Initial commit


<img alt='rebase foo on master' src='images/10-rebase-foo-on-master.svg' width=50% />

Let's sync `master` with `foo` before continuing. This is called a "fast-forward" merge.

In [23]:
git checkout master
git merge foo

Switched to branch 'master'
Updating 12844aa..0e9470e
Fast-forward
 bar.txt | 1 +
 foo.txt | 1 +
 2 files changed, 2 insertions(+)
 create mode 100644 bar.txt
 create mode 100644 foo.txt


In [24]:
git log --oneline

0e9470e (HEAD -> master, foo) Make commit on branch foo
e33d14f Make commit on branch bar
12844aa Make commit on branch master
553dbb7 Initial commit


![catch up](images/11-catch-up.svg)

## The Swiss Army knife: `git rebase --interactive`

Git gives you a ton of flexibility during a rebase. You can reorder commits, squash multiple commits into each other, and even run arbitrary commands. The interactive rebase gives you an opportunity to set the agenda before performing a rebase.

This is the single most useful command in git.

In [25]:
touch interactive.txt
git add interactive.txt

echo "1. fixup commits" >> interactive.txt
git commit interactive.txt -m 'Fixup commit'

echo "2. fixup commits" >> interactive.txt
git commit interactive.txt -m 'Fixup commit'

echo "3. fixup commits" >> interactive.txt
git commit interactive.txt -m 'Fixup commit'

[master 9ee57f3] Fixup commit
 1 file changed, 1 insertion(+)
 create mode 100644 interactive.txt
[master ecf548a] Fixup commit
 1 file changed, 1 insertion(+)
[master c0432ad] Fixup commit
 1 file changed, 1 insertion(+)


In [26]:
git log --oneline

c0432ad (HEAD -> master) Fixup commit
ecf548a Fixup commit
9ee57f3 Fixup commit
0e9470e (foo) Make commit on branch foo
e33d14f Make commit on branch bar
12844aa Make commit on branch master
553dbb7 Initial commit


![setup interactive rebase](images/12.0-interactive.svg)

In [27]:
git rebase -i foo

Successfully rebased and updated refs/heads/master.


![interactive rebase](images/12.1-interactive.png)

![interactive rebase](images/12.2-interactive.png)

In [28]:
git log --oneline

94cc657 (HEAD -> master) Fixup commit
0e9470e (foo) Make commit on branch foo
e33d14f Make commit on branch bar
12844aa Make commit on branch master
553dbb7 Initial commit


![interactive rebase](images/12.3-interactive.svg)

### Autosquash

When creating a commit, you can queue actions to be performed in a future rebase by using specially crafted commit messages. The `git commit --fixup` command is useful for creating these messages. The command `git rebase -i --autosquash` tells git to read through the commit messages and automatically set the agenda without manual intervention.

In [29]:
echo "4. fixup commits" >> interactive.txt
git commit interactive.txt --fixup HEAD

echo "5. fixup commits" >> interactive.txt
git commit interactive.txt --fixup HEAD~1

[master 60b357b] fixup! Fixup commit
 1 file changed, 1 insertion(+)
[master 5223ae5] fixup! Fixup commit
 1 file changed, 1 insertion(+)


In [30]:
git log --oneline

5223ae5 (HEAD -> master) fixup! Fixup commit
60b357b fixup! Fixup commit
94cc657 Fixup commit
0e9470e (foo) Make commit on branch foo
e33d14f Make commit on branch bar
12844aa Make commit on branch master
553dbb7 Initial commit


![autosquash setup](images/13.0-autosquash-setup.svg)

In [31]:
git rebase -i --autosquash foo

Successfully rebased and updated refs/heads/master.


![autosquash](images/13.1-autosquash.png)

In [32]:
git log --oneline

9560b6a (HEAD -> master) Fixup commit
0e9470e (foo) Make commit on branch foo
e33d14f Make commit on branch bar
12844aa Make commit on branch master
553dbb7 Initial commit


![autosquash setup](images/13.2-autosquash.svg)

## Other useful commands

- `revert` creates a new commit which undoes a previous commit.

- `cherry-pick` copies individual commits.

- `bisect` performs a binary search, using an arbitrary command as the criteria.
    - Useful to find which commit intorduced a bug.

## Aside: GitHub goodies with the `hub` command

To demonstrate git remotes, we first need to create a remote repository.

The `hub` command extends `git` with bindings to the GitHub API.

Install from your OS package manager. For me, that's brew:

```bash
brew install hub
```

Activate hub using a shell alias:

In [33]:
alias git=hub
git --version

git version 2.16.2
hub version 2.2.9


In [34]:
git help hub | head -n34

HUB(1)                            Hub Manual                            HUB(1)



NAME
       hub - git + hub = github

SYNOPSIS
       hub [--noop] COMMAND OPTIONS
       hub alias [-s] [SHELL]

   Expanded git commands:
       git init -g OPTIONS
       git clone [-p] OPTIONS [USER/]REPOSITORY DIRECTORY
       git remote add [-p] OPTIONS USER[/REPOSITORY]
       git remote set-url [-p] OPTIONS REMOTE-NAME USER[/REPOSITORY]
       git fetch USER-1,[USER-2,...]
       git checkout PULLREQ-URL [BRANCH]
       git merge PULLREQ-URL
       git cherry-pick GITHUB-REF
       git am GITHUB-URL
       git apply GITHUB-URL
       git push REMOTE-1,REMOTE-2,...,REMOTE-N [REF]
       git submodule add [-p] OPTIONS [USER/]REPOSITORY DIRECTORY

   Custom git commands:
       git create [NAME] [-p] [-d DESCRIPTION] [-h HOMEPAGE]
       git browse [-u] [[USER/]REPOSITORY] [SUBPAGE]
       git compare [-u] [USER] [[START...]END]
       git fork [--no-remote]
       git    pull-request   [-o|--browse]

## Publishing to GitHub

The `remote` command is used to tell our local repo about another repo with which we can share commits.

The `create` command (provided by `hub`) creates a remote repo for us on GitHub.

First, lets create a repo on GitHub to host our commits. Then let's tell our local repo about the upstream repo in the dsp-uga organization.

In [35]:
git create git-going -p

Updating origin
created repository: cbarrick/git-going


In [36]:
git remote add -p dsp-uga/git-going

In [37]:
git remote -v

dsp-uga	git@github.com:dsp-uga/git-going.git (fetch)
dsp-uga	git@github.com:dsp-uga/git-going.git (push)
origin	git@github.com:cbarrick/git-going.git (fetch)
origin	git@github.com:cbarrick/git-going.git (push)


In [38]:
git push origin master

Counting objects: 15, done.
Delta compression using up to 4 threads.
Compressing objects: 100% (9/9), done.
Writing objects: 100% (15/15), 1.24 KiB | 633.00 KiB/s, done.
Total 15 (delta 2), reused 0 (delta 0)
remote: Resolving deltas: 100% (2/2), done.
To github.com:cbarrick/git-going.git
 * [new branch]      master -> master


In [39]:
git push dsp-uga master -f

Counting objects: 15, done.
Delta compression using up to 4 threads.
Compressing objects: 100% (9/9), done.
Writing objects: 100% (15/15), 1.24 KiB | 105.00 KiB/s, done.
Total 15 (delta 2), reused 0 (delta 0)
remote: Resolving deltas: 100% (2/2), done.
To github.com:dsp-uga/git-going.git
 + fe24ed9...9560b6a master -> master (forced update)


### Useful remote commands
- `git fetch` downloads commits from a remote repo.
- `git push` uploads commits to a remote repo.
- `git pull` is essentially a fetch followed by a merge.

### Tips
- The command `git push -u SOMEREMOTE` lets you set a default remote for a branch so that you don't always have to specify.

## The GitHub flow

Git is turbo flexible for a ton of different workflows, but GitHub reccomends one particular workflow.

1. Every developer has her own fork of the repository.
    - The group may have an organization to host the cannonical repo, or pick a developer's fork to be cannonical.
2. Master on the cannonical repo is sacred.
    - NEVER rewrite history, e.g. `reset` or `rebase`.
3. For each new feature, create a branch.
4. Make some commits.
5. Open a pull request.
6. Discuss the changes.
7. Test.
8. Merge.

https://guides.github.com/introduction/flow/

### Protips
- Add remotes for all of your team members.
    - This makes is easy to checkout, review, and test before the merge.
    - Typically `origin` should refer to your own fork, the others should be named after the owner.
- Rebase your branch on master frequently.
    - This ensures that your code is compatible with the latest work.
- Never merge broken code.
    - When you push to a branch, the corresponding PR is updated.

## End

Thanks for coming out!

## Cleaning up

In [40]:
git rm bar.txt foo.txt interactive.txt master.txt
git commit -m 'Remove example files'

rm 'bar.txt'
rm 'foo.txt'
rm 'interactive.txt'
rm 'master.txt'
[master 791e227] Remove example files
 4 files changed, 9 deletions(-)
 delete mode 100644 bar.txt
 delete mode 100644 foo.txt
 delete mode 100644 interactive.txt
 delete mode 100644 master.txt


In [41]:
echo '# Git Going'                   > README.md
echo 'An intermediate git tutorial.' >> README.md

git add README.md
git commit -m 'Add README'

[master f623a46] Add README
 1 file changed, 2 insertions(+)
 create mode 100644 README.md


In [42]:
git add git-going.ipynb images
git commit -m 'Add notebook'

[master eaeb6f4] Add notebook
 21 files changed, 1296 insertions(+)
 create mode 100644 git-going.ipynb
 create mode 100644 images/00.0-master.svg
 create mode 100644 images/00.1-init.svg
 create mode 100644 images/01-foo.svg
 create mode 100644 images/02-bar.svg
 create mode 100644 images/04-commit-on-bar.svg
 create mode 100644 images/05-commit-on-bar-again.svg
 create mode 100644 images/06-reset.svg
 create mode 100644 images/07-commit-foo.svg
 create mode 100644 images/08-merge-bar-into-foo.svg
 create mode 100644 images/09-commit-on-master.svg
 create mode 100644 images/10-rebase-foo-on-master.svg
 create mode 100644 images/11-catch-up.svg
 create mode 100644 images/12.0-interactive-setup.svg
 create mode 100644 images/12.0-interactive.svg
 create mode 100644 images/12.1-interactive.png
 create mode 100644 images/12.1-interactive.svg
 create mode 100644 images/12.2-interactive.png
 create mode 100644 images/12.3-interactive.svg
 create mode 100644 images/13.0-autosquash-setup.svg


In [43]:
git push origin master
git push dsp-uga master

Counting objects: 27, done.
Delta compression using up to 4 threads.
Compressing objects: 100% (24/24), done.
Writing objects: 100% (27/27), 506.64 KiB | 7.68 MiB/s, done.
Total 27 (delta 14), reused 0 (delta 0)
remote: Resolving deltas: 100% (14/14), done.
To github.com:cbarrick/git-going.git
   9560b6a..eaeb6f4  master -> master
Counting objects: 27, done.
Delta compression using up to 4 threads.
Compressing objects: 100% (24/24), done.
Writing objects: 100% (27/27), 506.64 KiB | 13.69 MiB/s, done.
Total 27 (delta 14), reused 0 (delta 0)
remote: Resolving deltas: 100% (14/14), done.
To github.com:dsp-uga/git-going.git
   9560b6a..eaeb6f4  master -> master


## Maintenance
The diagrams in this notebook are drawn by hand, and thus the commit labels won't match when the notebook is re-rendered. Fortunately they are mostly SVG, e.g. plain text, so they can be updated from the shell:

```sh
sed -i s/OLDREF/NEWREF/g images/*.svg
```